# Aula 3 Trilha de Backend e Cloud - Melhorando a aplicação com Use Cases, DTOs e Documentação
Guilherme Kaidei - 2025



Agora que você já aprendeu a criar uma API básica com FastAPI e SQLAlchemy, vamos melhorar nossa aplicação, adcionando Use Cases, DTOs e uma melhor documentação com Swagger.

## Mudando a estrutrura do projeto

Até agora, a nossa aplicação tinha a seguinte estrutura:

```
app/
├── app.py
├── models/
├── repositories/
├── entities/
└── database/
```

Que já é uma estrutura melhor do que ter tudo em um único arquivo, mas ainda podemos melhorar. O principal problema dessa estrutura é que não temos uma separação das rotas, ou seja, tudo está em um único arquivo `app.py`, o que dificulta a manutenção e a escalabilidade do código. Para melhorar isso, vamos usar Use Cases, que são uma forma de organizar a lógica da aplicação em funções ou classes que representam um caso de uso específico. Por exemplo, podemos ter um Use Case para criar um usuário, outro para listar usuários, outro para atualizar usuários, etc. 

A nova estrutura do projeto ficará assim:
```
app/
├── app.py
├── models/
├── repositories/
├── entities/
├── database/
└── use_cases/
```

Entretanto, antes de criarmos os Use Cases, vamos mudar algumas coisas na nossa aplicação. Primeiro, vamos mudar o arquivo `app.py` para que ele apenas crie a aplicação FastAPI e importe as rotas de outros arquivos.

Mude o arquivo `app.py` para o seguinte:


In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import os
import glob
from importlib import import_module

app = FastAPI()

@app.get("/") # Aqui é definida uma rota GET para o endpoint raiz ("/") da aplicação. Assim, essa rota pode ser acessada para verficar se a aplicação está funcionando corretamente.
def test():
    return {"status": "OK v2 (3)"} 

app.add_middleware( # Aqui é adicionado um middleware de CORS (Cross-Origin Resource Sharing) à aplicação FastAPI.
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Aqui é definido o diretório de trabalho atual e o diretório onde estão os casos de uso (use_cases). Em seguida, são buscados todos os arquivos index.py dentro do diretório use_cases e seus subdiretórios. 
# Assim, para cada arquivo encontrado, o código tenta importar o módulo correspondente e, se o módulo tiver um atributo router, ele é incluído na aplicação FastAPI.
working_directory = os.path.dirname(os.path.abspath(__file__))
use_cases_directory = os.path.join(working_directory, "use_cases")
routes = glob.glob(os.path.join(use_cases_directory, "**/index.py"), recursive=True)

for route in routes:
    relative_path = os.path.relpath(route, working_directory)
    module_name = os.path.splitext(relative_path)[0].replace(os.path.sep, '.')

    try:
        print(f"Importing module: {module_name}")
        module = import_module(module_name)
        if hasattr(module, 'router'):
            app.include_router(module.router)
    except ModuleNotFoundError as e:
        print(f"Erro ao importar módulo {module_name}: {e}")



Em seguida, mude o arquivo `database/database.py` para o seguinte:

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, declarative_base
from dotenv import load_dotenv
import os

load_dotenv()

DATABASE_URL = os.getenv("DATABASE_URL", "sqlite:///./biblioteca.db")

engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

Aqui apenas movemos a função get_db para dentro do arquivo database.py, já que ela é responsável por criar a sessão do banco de dados.

Por último, mude a função delete do arquivo `repositories/base_repository.py` para o seguinte:

In [ ]:
def delete(self, id: int):
        entity = self.get_by_id(id)
        if entity:
            self.session.delete(entity)
            self.session.commit()
            return entity

Isso serve básicamente para dizermos que o objeto foi deletado.

## Criando os Use Cases
Vamos criar uma pasta chamada `use_cases` dentro da pasta `app`, e dentro dela, vamos criar pastas para cada tipo de caso de uso que queremos implementar, no nosso caso, vamos criar as pastas `authors` e `books`, contendo os casos de usos para cada uma dessas entidades. Dentro de cada pasta, vamos criar mais pastas para cada caso de uso, por exemplo, dentro da pasta `authors`, vamos criar as pastas `create_author_use_case`, `get_all_authors_use_case`, etc. e dentro de cada uma dessas pastas, vamos sempre ter dois arquivos: `index.py` e `algum_use_case.py`. O arquivo `index.py` será responsável por importar, instanciar e rotear o Use Case, enquanto o arquivo `algum_use_case.py` conterá a lógica do Use Case em si.

### Criando os casos de uso para Authors
Vamos começar criando os casos de uso para a entidade `Author`. Crie a seguinte estrutura de pastas e arquivos:

```app/
└── use_cases/
    └── authors/
        ├── create_author_use_case/
        │   ├── create_author_use_case.py
        │   └── index.py
        ├── get_all_authors_use_case/
        │   ├── get_all_authors_use_case.py
        │   └── index.py
        ├── get_author_by_id_use_case/
        │   ├── get_author_by_id_use_case.py
        │   └── index.py
        ├── update_author_use_case/
        │   ├── update_author_use_case.py
        │   └── index.py
        └── delete_author_use_case/
            ├── delete_author_use_case.py
            └── index.py
```

Vamos criar o primeiro caso de uso, que é o `create_author_use_case`. No arquivo `create_author_use_case.py`, vamos implementar a lógica para criar um autor:

In [ ]:
from repositories.author_repository import AuthorRepository
from entities.author import Author
from models.author_model import AuthorModel

class CreateAuthorUseCase:
    def __init__(self, author_repository: AuthorRepository): # O Use Case recebe uma instância do repositório de autores via injeção de dependência.
        self.author_repository = author_repository

    def execute(self, name: str, email: str) -> Author: # O método execute do Use Case cria um novo autor usando o repositório e retorna a entidade criada.
        author_model = AuthorModel(name=name, email=email)
        created_author = self.author_repository.add(author_model)
        return created_author


E no arquivo `index.py`, vamos importar o Use Case e criar a rota para ele:

In [ ]:
from fastapi import APIRouter, Depends
from repositories.author_repository import AuthorRepository
from database.database import get_db
from sqlalchemy.orm import Session
from entities.author import Author
from use_cases.authors.create_author_use_case.create_author_use_case import CreateAuthorUseCase
from pydantic import BaseModel

router = APIRouter()

class CreateAuthorRequest(BaseModel): # Define o modelo de dados esperado na requisição para criar um autor. (Vamos falar mais disso depois)
    name: str
    email: str

def get_use_case(db: Session = Depends(get_db)) -> CreateAuthorUseCase: # Função de dependência que cria uma instância do Use Case com o repositório necessário.
    author_repository = AuthorRepository(db)
    return CreateAuthorUseCase(author_repository)

@router.post("/authors", response_model=Author) # Define uma rota POST para o endpoint /authors que utiliza o Use Case para criar um novo autor.
def create_author(request: CreateAuthorRequest, use_case: CreateAuthorUseCase = Depends(get_use_case)):
    return use_case.execute(request.name, request.email)


Aqui, o `index.py` importa o `CreateAuthorUseCase`, instancia ele com o repositório de autores e cria a rota `/authors` que chama o método `execute` do Use Case.

Perceba que essa estrutura de pastas e arquivos pode parecer um pouco exagerada para uma aplicação simples, mas ela é muito útil para aplicações maiores e mais complexas, onde a organização do código é fundamental para a manutenção e escalabilidade da aplicação. Além disso, essa estrutura facilita a adição de novos casos de uso no futuro, sem precisar modificar muito o código existente.

Imagina fazer um aplicativo com 50 endpoints diferentes, e todos eles estarem em um único arquivo `app.py`. Seria um caos para manter e escalar a aplicação. Com essa estrutura, cada caso de uso fica isolado em sua própria pasta, facilitando a manutenção e a adição de novos casos de uso no futuro.

Vamos seguir o mesmo padrão para os outros casos de uso. Aqui estão os códigos para cada um dos arquivos restantes:

`delete_author_use_case/index.py`

In [ ]:
from fastapi import APIRouter, Depends, HTTPException
from repositories.author_repository import AuthorRepository
from database.database import get_db
from sqlalchemy.orm import Session
from use_cases.authors.delete_author_use_case.delete_author_use_case import DeleteAuthorUseCase

router = APIRouter()

def get_use_case(db: Session = Depends(get_db)) -> DeleteAuthorUseCase:
    author_repository = AuthorRepository(db)
    return DeleteAuthorUseCase(author_repository)

@router.delete("/authors/{author_id}")
def delete_author(author_id: int, use_case: DeleteAuthorUseCase = Depends(get_use_case)):
    success = use_case.execute(author_id)
    if not success:
        raise HTTPException(status_code=404, detail="Author not found")
    return {"message": "Author deleted successfully"}


`delete_author_use_case/delete_author_use_case.py`

In [ ]:
from repositories.author_repository import AuthorRepository

class DeleteAuthorUseCase:
    def __init__(self, author_repository: AuthorRepository):
        self.author_repository = author_repository

    def execute(self, author_id: int) -> bool:
        return self.author_repository.delete(author_id)

`get_all_authors_use_case/index.py`

In [ ]:
from fastapi import APIRouter, Depends
from repositories.author_repository import AuthorRepository
from database.database import get_db
from sqlalchemy.orm import Session
from models.author_model import AuthorModel
from entities.author import Author
from use_cases.authors.get_all_authors_use_case.get_all_authors_use_case import GetAllAuthorsUseCase

router = APIRouter()

def get_use_case(db: Session = Depends(get_db)) -> GetAllAuthorsUseCase:
    author_repository = AuthorRepository(db)
    return GetAllAuthorsUseCase(author_repository)

@router.get("/authors", response_model=list[Author])
def get_all_authors(use_case: GetAllAuthorsUseCase = Depends(get_use_case)):
    return use_case.execute()

`get_all_authors_use_case/get_all_authos_use_case.py`

In [ ]:
from repositories.author_repository import AuthorRepository
from entities.author import Author

class GetAllAuthorsUseCase:
    def __init__(self, author_repository: AuthorRepository):
        self.author_repository = author_repository

    def execute(self) -> list[Author]:
        authors = self.author_repository.get_all()
        return authors

`get_author_by_id_use_case/index.py`

In [ ]:
from fastapi import APIRouter, Depends, HTTPException
from repositories.author_repository import AuthorRepository
from database.database import get_db
from sqlalchemy.orm import Session
from entities.author import Author
from use_cases.authors.get_author_by_id_use_case.get_author_by_id_use_case import GetAuthorByIdUseCase

router = APIRouter()

def get_use_case(db: Session = Depends(get_db)) -> GetAuthorByIdUseCase:
    author_repository = AuthorRepository(db)
    return GetAuthorByIdUseCase(author_repository)

@router.get("/authors/{author_id}", response_model=Author)
def get_author_by_id(author_id: int, use_case: GetAuthorByIdUseCase = Depends(get_use_case)):
    author = use_case.execute(author_id)
    if not author:
        raise HTTPException(status_code=404, detail="Author not found")
    return author


`get_author_by_id_use_case/get_author_by_id_use_case.py`

In [ ]:
from repositories.author_repository import AuthorRepository
from entities.author import Author
from typing import Optional

class GetAuthorByIdUseCase:
    def __init__(self, author_repository: AuthorRepository):
        self.author_repository = author_repository

    def execute(self, author_id: int) -> Optional[Author]:
        author = self.author_repository.get_by_id(author_id)
        return author


`update_author_use_case/index.py`

In [ ]:
from fastapi import APIRouter, Depends, HTTPException
from repositories.author_repository import AuthorRepository
from database.database import get_db
from sqlalchemy.orm import Session
from entities.author import Author
from use_cases.authors.update_author_use_case.update_author_use_case import UpdateAuthorUseCase
from pydantic import BaseModel

router = APIRouter()

class UpdateAuthorRequest(BaseModel):
    name: str
    email: str

def get_use_case(db: Session = Depends(get_db)) -> UpdateAuthorUseCase:
    author_repository = AuthorRepository(db)
    return UpdateAuthorUseCase(author_repository)

@router.put("/authors/{author_id}", response_model=Author)
def update_author(author_id: int, request: UpdateAuthorRequest, use_case: UpdateAuthorUseCase = Depends(get_use_case)):
    author = use_case.execute(author_id, request.name, request.email)
    if not author:
        raise HTTPException(status_code=404, detail="Author not found")
    return author


`update_author_use_case/update_author_use_case.py`

In [ ]:
from repositories.author_repository import AuthorRepository
from entities.author import Author
from typing import Optional

class UpdateAuthorUseCase:
    def __init__(self, author_repository: AuthorRepository):
        self.author_repository = author_repository

    def execute(self, author_id: int, name: str, email: str) -> Optional[Author]:
        author = self.author_repository.get_by_id(author_id)
        if not author:
            return None
        
        author.name = name
        author.email = email
        updated_author = self.author_repository.update(author)
        return updated_author


Agora que temos todos os casos de uso para a entidade `Author`, tente criar os casos de uso para a entidade `Book` seguindo o mesmo padrão. Lembre-se de criar uma pasta books para seus casos de uso dessa entidade. O gabarito está aqui embaixo:

GABARITO:

`create_book_use_case/index.py`

```python
from fastapi import APIRouter, Depends
from repositories.book_repository import BookRepository
from database.database import get_db
from sqlalchemy.orm import Session
from entities.book import Book
from use_cases.books.create_book_use_case.create_book_use_case import CreateBookUseCase
from pydantic import BaseModel
from typing import Optional

router = APIRouter()

class CreateBookRequest(BaseModel):
    title: str
    isbn: str

def get_use_case(db: Session = Depends(get_db)) -> CreateBookUseCase:
    book_repository = BookRepository(db)
    return CreateBookUseCase(book_repository)

@router.post("/books", response_model=Book)
def create_book(request: CreateBookRequest, use_case: CreateBookUseCase = Depends(get_use_case)):
    return use_case.execute(request.title, request.isbn)
```


`create_book_use_case/create_book_use_case.py`

```python
from repositories.book_repository import BookRepository
from entities.book import Book
from models.book_model import BookModel
from typing import Optional

class CreateBookUseCase:
    def __init__(self, book_repository: BookRepository):
        self.book_repository = book_repository

    def execute(self, title: str, isbn: str) -> Book:
        book_model = BookModel(title=title,  isbn=isbn)
        created_book = self.book_repository.add(book_model)
        return created_book
```


`delete_book_use_case/index.py`

```python
from fastapi import APIRouter, Depends, HTTPException
from repositories.book_repository import BookRepository
from database.database import get_db
from sqlalchemy.orm import Session
from use_cases.books.delete_book_use_case.delete_book_use_case import DeleteBookUseCase

router = APIRouter()

def get_use_case(db: Session = Depends(get_db)) -> DeleteBookUseCase:
    book_repository = BookRepository(db)
    return DeleteBookUseCase(book_repository)

@router.delete("/books/{book_id}")
def delete_book(book_id: int, use_case: DeleteBookUseCase = Depends(get_use_case)):
    success = use_case.execute(book_id)
    if not success:
        raise HTTPException(status_code=404, detail="Book not found")
    return {"message": "Book deleted successfully"}
```


`delete_book_use_case/delete_book_use_case.py`

```python
from repositories.book_repository import BookRepository

class DeleteBookUseCase:
    def __init__(self, book_repository: BookRepository):
        self.book_repository = book_repository

    def execute(self, book_id: int) -> bool:
        return self.book_repository.delete(book_id)
```


`get_all_books_use_case/index.py`

```python
from fastapi import APIRouter, Depends
from repositories.book_repository import BookRepository
from database.database import get_db
from sqlalchemy.orm import Session
from entities.book import Book
from use_cases.books.get_all_books_use_case.get_all_books_use_case import GetAllBooksUseCase

router = APIRouter()

def get_use_case(db: Session = Depends(get_db)) -> GetAllBooksUseCase:
    book_repository = BookRepository(db)
    return GetAllBooksUseCase(book_repository)

@router.get("/books", response_model=list[Book])
def get_all_books(use_case: GetAllBooksUseCase = Depends(get_use_case)):
    return use_case.execute()
```


`get_all_books_use_case/get_all_books_use_case.py`
```python
from repositories.book_repository import BookRepository
from entities.book import Book

class GetAllBooksUseCase:
    def __init__(self, book_repository: BookRepository):
        self.book_repository = book_repository

    def execute(self) -> list[Book]:
        books = self.book_repository.get_all()
        return books
```


`get_book_by_id_use_case/index.py`

```python
from fastapi import APIRouter, Depends, HTTPException
from repositories.book_repository import BookRepository
from database.database import get_db
from sqlalchemy.orm import Session
from entities.book import Book
from use_cases.books.get_book_by_id_use_case.get_book_by_id_use_case import GetBookByIdUseCase

router = APIRouter()

def get_use_case(db: Session = Depends(get_db)) -> GetBookByIdUseCase:
    book_repository = BookRepository(db)
    return GetBookByIdUseCase(book_repository)

@router.get("/books/{book_id}", response_model=Book)
def get_book_by_id(book_id: int, use_case: GetBookByIdUseCase = Depends(get_use_case)):
    book = use_case.execute(book_id)
    if not book:
        raise HTTPException(status_code=404, detail="Book not found")
    return book
```


`get_book_by_id_use_case/get_book_by_id_use_case.py`

```python
from repositories.book_repository import BookRepository
from entities.book import Book
from typing import Optional

class GetBookByIdUseCase:
    def __init__(self, book_repository: BookRepository):
        self.book_repository = book_repository

    def execute(self, book_id: int) -> Optional[Book]:
        book = self.book_repository.get_by_id(book_id)
        return book
```


`update_book_use_case/index.py`

```python
from fastapi import APIRouter, Depends, HTTPException
from repositories.book_repository import BookRepository
from database.database import get_db
from sqlalchemy.orm import Session
from entities.book import Book
from use_cases.books.update_book_use_case.update_book_use_case import UpdateBookUseCase
from pydantic import BaseModel
from typing import Optional

router = APIRouter()

class UpdateBookRequest(BaseModel):
    title: str
    isbn: str

def get_use_case(db: Session = Depends(get_db)) -> UpdateBookUseCase:
    book_repository = BookRepository(db)
    return UpdateBookUseCase(book_repository)

@router.put("/books/{book_id}", response_model=Book)
def update_book(book_id: int, request: UpdateBookRequest, use_case: UpdateBookUseCase = Depends(get_use_case)):
    book = use_case.execute(book_id, request.title, request.isbn)
    if not book:
        raise HTTPException(status_code=404, detail="Book not found")
    return book
```


`update_book_use_case/update_book_use_case.py`

```python
from repositories.book_repository import BookRepository
from entities.book import Book
from typing import Optional

class UpdateBookUseCase:
    def __init__(self, book_repository: BookRepository):
        self.book_repository = book_repository

    def execute(self, book_id: int, title: str, isbn: str) -> Optional[Book]:
        book = self.book_repository.get_by_id(book_id)
        if not book:
            return None
        
        book.title = title
        book.isbn = isbn
        updated_book = self.book_repository.update(book)
        return updated_book
```

Agora que temos nossos casos de usos, podemos testar nossa aplicação para ver se tudo está funcionando corretamente. Rode o comando `fastapi run app/app.py` e acesse o Swagger em `http://localhost:8000/docs`. 

Com o swagger aberto, perceba uma coisa: Na rota de criação de autores, o Swagger está pedindo um body com os campos `name` e `email`, entretando, a nossa entidade `Author` também tem um campo `id` e um campo `livros`. 

Isso acontece porque no nosso caso de uso de criação de autores, nós criamos uma classe `CreateAuthorRequest` que define exatamente os campos que queremos receber no corpo da requisição. 

```python
class CreateAuthorRequest(BaseModel):
    name: str
    email: str
```

Essa classe está agindo como um DTO (Data Transfer Object)

### O que são DTOs?
DTOs (Data Transfer Objects) são objetos simples que são usados para transferir dados entre diferentes partes de um sistema, como entre a camada de apresentação (API) e a camada de negócio (Use Cases). Eles geralmente não contêm lógica de negócio, apenas dados.

Isso é útil por várias razões:
1. **Separação de preocupações**: DTOs ajudam a separar a estrutura dos dados que a API expõe da estrutura dos dados que a camada de negócio usa internamente. Isso permite que você mude a estrutura interna sem afetar a API pública.
2. **Validação**: DTOs podem ser usados para validar os dados de entrada.
3. **Documentação**: DTOs ajudam a documentar claramente quais dados são esperados em cada endpoint da API. Pense por exemplo que provavelmente você não vai precisar que o usuário informe o ID do autor na hora de criar um novo autor, já que esse ID é gerado automaticamente pelo banco de dados. Então faz sentido que o DTO de criação de autores não tenha o campo ID.
4. **Segurança**: DTOs podem ajudar a evitar a exposição acidental de dados sensíveis que não deveriam ser expostos pela API.

Apesar dos DTOs estarem funcionando nesse formato, por uma questão de organização do código, é interessante que eles fiquem em um arquivo separado. Crie uma pasta chamada `dtos` dentro da pasta `app`, e dentro dela, crie os arquivos `author_dto.py` e `book_dto.py`. Mova as classes `CreateAuthorRequest`, `UpdateAuthorRequest`, `CreateBookRequest` e `UpdateBookRequest` para esses arquivos, respectivamente. Depois, importe essas classes nos arquivos `index.py` dos casos de uso correspondentes.

Agora, a estrutura do projeto ficará assim:

```app/
├── app.py
├── models/
├── repositories/
├── entities/
├── database/
├── dtos/
│   ├── author_dto.py
│   └── book_dto.py
└── use_cases/
```

Agora, vamos para outra funcionalidade que ainda não implementamos: o relacionamento entre autores e livros. Para isso, vamos seguir os mesmos passos de sempre, criar um caso de uso para adicionar um livro a um autor, um caso de uso para remover um livro de um autor, e um caso de uso para listar todos os livros de um autor. Poderiamos fazer isso no caso de uso de update, mas por uma questão de facilidade faremos casos de uso separados.

Entretanto, antes de criarmos esses casos de uso vamos parar e olhar as nossas entidades `Author` e `Book`. Atualmente, elas estão assim:

```python
from pydantic import BaseModel, EmailStr, Field
from typing import List, Optional

class Author(BaseModel):
    """
    Entidade Author usando Pydantic
    Representa um autor no domínio da aplicação
    """
    id: Optional[int] = None
    name: str = Field(..., min_length=1, max_length=255)
    email: EmailStr
    books: List['Book'] = Field(default_factory=list)

    class Config:
        # Permite conversão de objetos SQLAlchemy
        from_attributes = True

    def __str__(self):
        return f"Author: {self.name} ({self.email})"

# Resolve forward references
from .book import Book
Author.model_rebuild()
```

```python
from pydantic import BaseModel, Field
from typing import List, Optional

class Book(BaseModel):
    id: Optional[int] = None
    title: str = Field(..., min_length=1, max_length=255)
    isbn: Optional[str] = Field(..., min_length=10, max_length=17)
    authors: List['Author'] = Field(default_factory=list)
    
    class Config:
        from_attributes = True
        
    def __str__(self):
        return f"Book: {self.title} ({self.isbn})"
    
# Resolve forward references
from .author import Author
Book.model_rebuild()
```

Perceba que a entidade `Author` tem uma lista de livros, e a entidade `Book` tem uma lista de autores. Isso representa um relacionamento muitos-para-muitos entre autores e livros, ou seja, um autor pode ter vários livros, e um livro pode ter vários autores.

Apesar desse relacionamento estar certo, você consegue perceber um problema? Olhe para o que acontece quando você tenta listar todos os autores ou livros.

Ao listar qualquer autor ou livro, o repositório retorna a entidade base com todos os seus relacionamentos. Ou seja, ao listar um autor, você também está listando todos os livros daquele autor, e para cada livro, você está listando todos os autores daquele livro, e para cada autor, você está listando todos os livros daquele autor, e assim por diante. Isso pode levar a um estouro de memória (RecursionError: maximum recursion depth exceeded).

Mas como podemos resolver isso? A resposta são os DTOs. Podemos criar DTOs específicos para listar autores e livros, que terão os campos necessários para a listagem, sem incluir os relacionamentos que podem causar o estouro de memória. Nesse caso, o que vamos fazer é criar dois novos DTOs: `AuthorResponse` e `BookResponse`, que terão os campos necessários para a listagem, sem incluir os relacionamentos.

Dentro do arquivo `author_dto.py`, crie a classe `AuthorResponse` e a classe `BookSummary`:



In [ ]:
from typing import List # Adicione esse import


class BookSummary(BaseModel):
    id: int
    title: str
    isbn: str

    class Config:
        from_attributes = True

class AuthorResponse(BaseModel):
    id: int
    name: str
    email: str
    books: List[BookSummary] = []

    class Config:
        from_attributes = True

Com esse DTO, quando listarmos os autores, não teremos mais o problema de estouro de memória, pois o DTO `AuthorResponse` não inclui a lista de livros com todos os detalhes, apenas um resumo dos livros (ID e título).

Agora, mude o caso de uso `get_all_authors_use_case.py` para usar o DTO `AuthorResponse` como resposta:

In [ ]:
from repositories.author_repository import AuthorRepository
from dtos.author_dto import AuthorResponse 

class GetAllAuthorsUseCase:
    def __init__(self, author_repository: AuthorRepository):
        self.author_repository = author_repository

    def execute(self) -> list[AuthorResponse]:
        authors = self.author_repository.get_all() 
        return [AuthorResponse.model_validate(author) for author in authors]


E o arquivo `index.py` também:

In [ ]:
from fastapi import APIRouter, Depends
from repositories.author_repository import AuthorRepository
from database.database import get_db
from sqlalchemy.orm import Session
from dtos.author_dto import AuthorResponse 

from use_cases.authors.get_all_authors_use_case.get_all_authors_use_case import GetAllAuthorsUseCase

router = APIRouter()

def get_use_case(db: Session = Depends(get_db)) -> GetAllAuthorsUseCase:
    author_repository = AuthorRepository(db)
    return GetAllAuthorsUseCase(author_repository)

@router.get("/authors", response_model=list[AuthorResponse])
def get_all_authors(use_case: GetAllAuthorsUseCase = Depends(get_use_case)):
    return use_case.execute()

Além disso, precisamos usar esse mesmo DTO em todos os casos de uso que retornam autores, como `get_author_by_id_use_case.py` e `update_author_use_case.py` (Os arquivos de index também precisam ser atualizados). Faça isso para todos os casos de uso que retornam autores. (No nosso caso, não precisamos fazer isso para o caso de uso de criação, pois não criamos autores com livros diretamente).

`create_author_use_case/index.py`: 

In [ ]:
from fastapi import APIRouter, Depends
from repositories.author_repository import AuthorRepository
from database.database import get_db
from sqlalchemy.orm import Session
from entities.author import Author
from use_cases.authors.create_author_use_case.create_author_use_case import CreateAuthorUseCase
from dtos.author_dto import CreateAuthorRequest

router = APIRouter()

def get_use_case(db: Session = Depends(get_db)) -> CreateAuthorUseCase:
    author_repository = AuthorRepository(db)
    return CreateAuthorUseCase(author_repository)

@router.post("/authors", response_model=Author)
def create_author(request: CreateAuthorRequest, use_case: CreateAuthorUseCase = Depends(get_use_case)):
    return use_case.execute(request.name, request.email)


`create_author_use_case/create_author_use_case.py`: 


In [ ]:
from repositories.author_repository import AuthorRepository
from entities.author import Author
from models.author_model import AuthorModel

class CreateAuthorUseCase:
    def __init__(self, author_repository: AuthorRepository):
        self.author_repository = author_repository

    def execute(self, name: str, email: str) -> Author:
        author_model = AuthorModel(name=name, email=email)
        
        created_author = self.author_repository.add(author_model)
        return created_author


`delete_author_use_case/index.py`:


In [ ]:
from fastapi import APIRouter, Depends, HTTPException
from repositories.author_repository import AuthorRepository
from database.database import get_db
from sqlalchemy.orm import Session
from use_cases.authors.delete_author_use_case.delete_author_use_case import DeleteAuthorUseCase
from dtos.author_dto import AuthorResponse 


router = APIRouter()

def get_use_case(db: Session = Depends(get_db)) -> DeleteAuthorUseCase:
    author_repository = AuthorRepository(db)
    return DeleteAuthorUseCase(author_repository)

@router.delete("/authors/{author_id}")
def delete_author(author_id: int, use_case: DeleteAuthorUseCase = Depends(get_use_case), response_model=AuthorResponse): 
    success = use_case.execute(author_id)
    if not success:
        raise HTTPException(status_code=404, detail="Author not found")
    return {"message": "Author deleted successfully"}


`delete_author_use_case/delete_author_use_case.py`:


In [ ]:
from repositories.author_repository import AuthorRepository
from dtos.author_dto import AuthorResponse 


class DeleteAuthorUseCase:
    def __init__(self, author_repository: AuthorRepository):
        self.author_repository = author_repository

    def execute(self, author_id: int) -> AuthorResponse | None:
        author = self.author_repository.get_by_id(author_id)
        if not author:
            return None
        response = self.author_repository.delete(author_id)
        return AuthorResponse.model_validate(response)


`get_author_by_id_use_case/index.py`:


In [ ]:
from fastapi import APIRouter, Depends, HTTPException
from repositories.author_repository import AuthorRepository
from database.database import get_db
from sqlalchemy.orm import Session
from dtos.author_dto import AuthorResponse 
from use_cases.authors.get_author_by_id_use_case.get_author_by_id_use_case import GetAuthorByIdUseCase

router = APIRouter()

def get_use_case(db: Session = Depends(get_db)) -> GetAuthorByIdUseCase:
    author_repository = AuthorRepository(db)
    return GetAuthorByIdUseCase(author_repository)

@router.get("/authors/{author_id}", response_model=AuthorResponse)
def get_author_by_id(author_id: int, use_case: GetAuthorByIdUseCase = Depends(get_use_case)):
    author = use_case.execute(author_id)
    if not author:
        raise HTTPException(status_code=404, detail="Author not found")
    return author


`get_author_by_id_use_case/get_author_by_id_use_case.py`:


In [ ]:
from repositories.author_repository import AuthorRepository
from dtos.author_dto import AuthorResponse 
from typing import Optional

class GetAuthorByIdUseCase:
    def __init__(self, author_repository: AuthorRepository):
        self.author_repository = author_repository

    def execute(self, author_id: int) -> Optional[AuthorResponse]:
        author = self.author_repository.get_by_id(author_id)
        return AuthorResponse.model_validate(author)


`update_author_use_case/index.py`:


In [ ]:
from fastapi import APIRouter, Depends, HTTPException
from repositories.author_repository import AuthorRepository
from database.database import get_db
from sqlalchemy.orm import Session
from dtos.author_dto import AuthorResponse 
from use_cases.authors.update_author_use_case.update_author_use_case import UpdateAuthorUseCase
from dtos.author_dto import UpdateAuthorRequest

router = APIRouter()

def get_use_case(db: Session = Depends(get_db)) -> UpdateAuthorUseCase:
    author_repository = AuthorRepository(db)
    return UpdateAuthorUseCase(author_repository)

@router.put("/authors/{author_id}", response_model= AuthorResponse)
def update_author(author_id: int, request: UpdateAuthorRequest, use_case: UpdateAuthorUseCase = Depends(get_use_case)):
    author = use_case.execute(author_id, request.name, request.email)
    if not author:
        raise HTTPException(status_code=404, detail="Author not found")
    return author


`update_author_use_case/update_author_use_case.py`:

In [ ]:
from repositories.author_repository import AuthorRepository
from dtos.author_dto import AuthorResponse 
from typing import Optional

class UpdateAuthorUseCase:
    def __init__(self, author_repository: AuthorRepository):
        self.author_repository = author_repository

    def execute(self, author_id: int, name: str, email: str) -> Optional[AuthorResponse]:
        author = self.author_repository.get_by_id(author_id)
        if not author:
            return None
        
        author.name = name
        author.email = email
        updated_author = self.author_repository.update(author)
        return AuthorResponse.model_validate(updated_author)


Agora que temos o DTO de resposta para autores, tente criar sozinho o DTO de resposta para livros, seguindo o mesmo padrão. Lembre-se de criar uma classe `AuthorSummary` dentro do arquivo `book_dto.py`, que terá apenas os campos `id`, `name` e `email`.

GABARITO:

```python
class AuthorSummary(BaseModel):
    id: int
    name: str
    email: str

    class Config:
        from_attributes = True

class BookResponse(BaseModel):
    id: int
    title: str
    isbn: str
    authors: List[AuthorSummary] = []

    class Config:
        from_attributes = True
```

Agora que temos o DTO de resposta para livros, faça as mesmas alterações que fizemos para os autores nos casos de uso de livros, para que eles retornem o DTO `BookResponse` ao invés da entidade `Book`.

GABARITO:
`create_book_use_case/index.py`

```python
from fastapi import APIRouter, Depends
from repositories.book_repository import BookRepository
from database.database import get_db
from sqlalchemy.orm import Session
from dtos.book_dto import BookResponse
from use_cases.books.create_book_use_case.create_book_use_case import CreateBookUseCase
from dtos.book_dto import CreateBookRequest

router = APIRouter()

def get_use_case(db: Session = Depends(get_db)) -> CreateBookUseCase:
    book_repository = BookRepository(db)
    return CreateBookUseCase(book_repository)

@router.post("/books", response_model=BookResponse)
def create_book(request: CreateBookRequest, use_case: CreateBookUseCase = Depends(get_use_case)):
    return use_case.execute(request.title, request.isbn)
```


`create_book_use_case/create_book_use_case.py`

```python
from repositories.book_repository import BookRepository
from dtos.book_dto import BookResponse
from models.book_model import BookModel
from typing import Optional

class CreateBookUseCase:
    def __init__(self, book_repository: BookRepository):
        self.book_repository = book_repository

    def execute(self, title: str, isbn: str) -> BookResponse:
        book_model = BookModel(title=title,  isbn=isbn)
        created_book = self.book_repository.add(book_model)
        return BookResponse.model_validate(created_book)
```


`delete_book_use_case/index.py`

```python
from fastapi import APIRouter, Depends, HTTPException
from repositories.book_repository import BookRepository
from database.database import get_db
from sqlalchemy.orm import Session
from use_cases.books.delete_book_use_case.delete_book_use_case import DeleteBookUseCase

router = APIRouter()

def get_use_case(db: Session = Depends(get_db)) -> DeleteBookUseCase:
    book_repository = BookRepository(db)
    return DeleteBookUseCase(book_repository)

@router.delete("/books/{book_id}")
def delete_book(book_id: int, use_case: DeleteBookUseCase = Depends(get_use_case)):
    success = use_case.execute(book_id)
    if not success:
        raise HTTPException(status_code=404, detail="Book not found")
    return {"message": "Book deleted successfully"}
```


`delete_book_use_case/delete_book_use_case.py`

```python
from repositories.book_repository import BookRepository
from dtos.book_dto import BookResponse

class DeleteBookUseCase:
    def __init__(self, book_repository: BookRepository):
        self.book_repository = book_repository

    def execute(self, book_id: int) -> bool:
        book = self.book_repository.get(book_id)
        if not book:
            return None
        response = self.book_repository.delete(book_id)
        return BookResponse.model_validate(response)
```


`get_all_books_use_case/index.py`

```python
from fastapi import APIRouter, Depends
from repositories.book_repository import BookRepository
from database.database import get_db
from sqlalchemy.orm import Session
from dtos.book_dto import BookResponse
from use_cases.books.get_all_books_use_case.get_all_books_use_case import GetAllBooksUseCase

router = APIRouter()

def get_use_case(db: Session = Depends(get_db)) -> GetAllBooksUseCase:
    book_repository = BookRepository(db)
    return GetAllBooksUseCase(book_repository)

@router.get("/books", response_model=list[BookResponse])
def get_all_books(use_case: GetAllBooksUseCase = Depends(get_use_case)):
    return use_case.execute()
```


`get_all_books_use_case/get_all_books_use_case.py`

```python
from repositories.book_repository import BookRepository
from dtos.book_dto import BookResponse

class GetAllBooksUseCase:
    def __init__(self, book_repository: BookRepository):
        self.book_repository = book_repository

    def execute(self) -> list[BookResponse]:
        books = self.book_repository.get_all()
        return [BookResponse.model_validate(book) for book in books]
```


`get_book_by_id_use_case/index.py`

```python
from fastapi import APIRouter, Depends, HTTPException
from repositories.book_repository import BookRepository
from database.database import get_db
from sqlalchemy.orm import Session
from dtos.book_dto import BookResponse
from use_cases.books.get_book_by_id_use_case.get_book_by_id_use_case import GetBookByIdUseCase

router = APIRouter()

def get_use_case(db: Session = Depends(get_db)) -> GetBookByIdUseCase:
    book_repository = BookRepository(db)
    return GetBookByIdUseCase(book_repository)

@router.get("/books/{book_id}", response_model=BookResponse)
def get_book_by_id(book_id: int, use_case: GetBookByIdUseCase = Depends(get_use_case)):
    book = use_case.execute(book_id)
    if not book:
        raise HTTPException(status_code=404, detail="Book not found")
    return book
```


`get_book_by_id_use_case/get_book_by_id_use_case.py`

```python
from repositories.book_repository import BookRepository
from dtos.book_dto import BookResponse
from typing import Optional

class GetBookByIdUseCase:
    def __init__(self, book_repository: BookRepository):
        self.book_repository = book_repository

    def execute(self, book_id: int) -> Optional[BookResponse]:
        book = self.book_repository.get_by_id(book_id)
        return BookResponse.model_validate(book) if book else None
```


`update_book_use_case/index.py`

```python
from fastapi import APIRouter, Depends, HTTPException
from repositories.book_repository import BookRepository
from database.database import get_db
from sqlalchemy.orm import Session
from dtos.book_dto import BookResponse
from use_cases.books.update_book_use_case.update_book_use_case import UpdateBookUseCase
from dtos.book_dto import UpdateBookRequest

router = APIRouter()


def get_use_case(db: Session = Depends(get_db)) -> UpdateBookUseCase:
    book_repository = BookRepository(db)
    return UpdateBookUseCase(book_repository)

@router.put("/books/{book_id}", response_model=BookResponse)
def update_book(book_id: int, request: UpdateBookRequest, use_case: UpdateBookUseCase = Depends(get_use_case)):
    book = use_case.execute(book_id, request.title, request.isbn)
    if not book:
        raise HTTPException(status_code=404, detail="Book not found")
    return book
```

`update_book_use_case/update_book_use_case.py`

```python
from repositories.book_repository import BookRepository
from dtos.book_dto import BookResponse
from typing import Optional

class UpdateBookUseCase:
    def __init__(self, book_repository: BookRepository):
        self.book_repository = book_repository

    def execute(self, book_id: int, title: str, isbn: str) -> Optional[BookResponse]:
        book = self.book_repository.get_by_id(book_id)
        if not book:
            return None
        
        book.title = title
        book.isbn = isbn
        updated_book = self.book_repository.update(book)
        return BookResponse.model_validate(updated_book) if updated_book else None
```

Agora que resolvemos o problema do estouro de memória, podemos voltar a implementar os casos de uso para gerenciar o relacionamento entre autores e livros.

No nosso caso vamos criar dois casos de uso: um para adicionar um livro a um autor, e outro para remover um livro de um autor. Vamos fazer isso na pasta `authors`, já que faz mais sentido gerenciar os livros a partir do autor:

`add_book_to_author_use_case/index.py`: 

In [ ]:
from fastapi import APIRouter, Depends, HTTPException
from repositories.author_repository import AuthorRepository
from repositories.book_repository import BookRepository
from database.database import get_db
from sqlalchemy.orm import Session
from dtos.author_dto import AuthorResponse 
from use_cases.authors.add_book_to_author_use_case.add_book_to_author_use_case import AddBookToAuthorUseCase

router = APIRouter()

def get_use_case(db: Session = Depends(get_db)) -> AddBookToAuthorUseCase:
    author_repository = AuthorRepository(db)
    book_repository = BookRepository(db)
    return AddBookToAuthorUseCase(author_repository, book_repository)

@router.post("/authors/{author_id}/books/{book_id}", response_model=AuthorResponse)
def add_book_to_author(author_id: int, book_id: int, use_case: AddBookToAuthorUseCase = Depends(get_use_case)):
    author = use_case.execute(author_id, book_id)
    if not author:
        raise HTTPException(status_code=404, detail="Author or Book not found")
    return author


`add_book_to_author_use_case/add_book_to_author_use_case.py`:

In [ ]:
from repositories.author_repository import AuthorRepository
from repositories.book_repository import BookRepository
from typing import Optional
from dtos.author_dto import AuthorResponse 


class AddBookToAuthorUseCase:
    def __init__(self, author_repository: AuthorRepository, book_repository: BookRepository):
        self.author_repository = author_repository
        self.book_repository = book_repository

    def execute(self, author_id: int, book_id: int) -> Optional[AuthorResponse]:
        author = self.author_repository.get_by_id(author_id)
        book = self.book_repository.get_by_id(book_id)
        
        if not author or not book:
            return None

        if book not in author.books:
            author.books.append(book)
            self.author_repository.update(author)

        return AuthorResponse.model_validate(author)



Perceba que nesse caso de uso a lógica é apenas fazer um update no autor, adicionando o livro na lista de livros do autor. Isso porque o relacionamento muitos-para-muitos já está sendo gerenciado pelo SQLAlchemy, então não precisamos fazer nada muito complexo.

Agora, crie o caso de uso para remover um livro de um autor, seguindo o mesmo padrão

GABARITO:

`remove_book_from_author_use_case/index.py`:

```python
from fastapi import APIRouter, Depends, HTTPException
from repositories.author_repository import AuthorRepository
from repositories.book_repository import BookRepository
from database.database import get_db
from sqlalchemy.orm import Session
from dtos.author_dto import AuthorResponse 
from use_cases.authors.remove_book_from_author_use_case.remove_book_from_author_use_case import RemoveBookFromAuthorUseCase

router = APIRouter()

def get_use_case(db: Session = Depends(get_db)) -> RemoveBookFromAuthorUseCase:
    author_repository = AuthorRepository(db)
    book_repository = BookRepository(db)
    return RemoveBookFromAuthorUseCase(author_repository, book_repository)

@router.delete("/authors/{author_id}/books/{book_id}", response_model=AuthorResponse)
def remove_book_from_author(author_id: int, book_id: int, use_case: RemoveBookFromAuthorUseCase = Depends(get_use_case)):
    author = use_case.execute(author_id, book_id)
    if not author:
        raise HTTPException(status_code=404, detail="Author or Book not found")
    return author
```

`remove_book_from_author_use_case/remove_book_from_author_use_case.py`:

```python
from repositories.author_repository import AuthorRepository
from repositories.book_repository import BookRepository
from typing import Optional
from dtos.author_dto import AuthorResponse 


class RemoveBookFromAuthorUseCase:
    def __init__(self, author_repository: AuthorRepository, book_repository: BookRepository):
        self.author_repository = author_repository
        self.book_repository = book_repository

    def execute(self, author_id: int, book_id: int) -> Optional[AuthorResponse]:
        author = self.author_repository.get_by_id(author_id)
        book = self.book_repository.get_by_id(book_id)
        
        if not author or not book:
            return None
        
        # Check if the book is associated with the author
        if book in author.books:
            author.books.remove(book)
            self.author_repository.update(author)

        return AuthorResponse.model_validate(author)
```


Parabéns! Você implementou com sucesso uma API RESTful usando FastAPI e SQLAlchemy, seguindo boas práticas de Clean Architecture, incluindo o uso de Use Cases e DTOs para organizar a lógica da aplicação e melhorar a documentação da API. Agora só falta você testar os novos casos de uso que você criou para gerenciar o relacionamento entre autores e livros. Rode o comando `fastapi run app/app.py` e acesse o Swagger em `http://localhost:8000/docs` para testar as novas rotas.

Caso queira, tente implementar mais funcionalidades por conta própria, como por exemplo, um caso de uso para listar todos os livros de um autor, ou um caso de uso para listar todos os autores de um livro. Use a mesma estrutura que você já aprendeu aqui, criando novos casos de uso, DTOs e rotas conforme necessário.

Entretanto, o handout ainda não acabou. Vamos fazer uma última melhoria na nossa aplicação: Documentação.

## Por que documentar a API?

Documentar a API é uma prática essencial no desenvolvimento de software, especialmente quando se trata de APIs RESTful. Uma boa documentação traz diversos benefícios, tanto para os desenvolvedores que irão consumir a API quanto para os próprios desenvolvedores que estão criando e mantendo a API. Aqui estão alguns dos principais motivos para documentar a API:

1. **Facilita o uso da API**: Uma documentação clara e concisa ajuda os desenvolvedores a entender como usar a API corretamente, reduzindo a curva de aprendizado e evitando erros.

2. **Melhora a manutenção**: Com uma boa documentação, é mais fácil para os desenvolvedores que estão mantendo a API entenderem como ela funciona e quais são suas dependências.

3. **Aumenta a colaboração**: Quando a API é bem documentada, outros desenvolvedores podem contribuir mais facilmente para o projeto, seja corrigindo bugs, adicionando novas funcionalidades ou melhorando a documentação.

4. **Serve como referência**: A documentação da API pode servir como um guia de referência para os desenvolvedores, ajudando-os a lembrar como usar diferentes endpoints e quais parâmetros são necessários.

5. **Facilita a integração**: Para APIs que serão consumidas por terceiros, uma boa documentação é fundamental para garantir que os desenvolvedores externos consigam integrar suas aplicações de forma rápida e eficiente.

Para o contexto da Insper Jr. é bem fácil de perceber onde esses pontos se encaixam:

- **Facilita a entrada de novos membros em projetos**: Novos membros da equipe podem ter dificuldades para entender como a API funciona sem uma documentação adequada. Isso pode levar a erros e retrabalho, o que é problemático principalmente quando olhamos a alta rotatividade de membros da empresa.
- **Entrega de projetos para clientes**: Quando a Insper Jr. entrega um projeto para um cliente, é essencial que o cliente, ou algum funcionário dele, consiga entender como usar a API. Uma boa documentação ajuda a garantir que o cliente possa utilizar a API de forma eficaz, aumentando a satisfação do cliente e reduzindo o suporte necessário.

## Documentando a API com FastAPI

A primeira coisa que podemos fazer para documentar nossa API é usar as docstrings. Docstrings são strings de documentação que são colocadas logo abaixo da definição de uma função, classe ou módulo. Elas são usadas para descrever o que a função, classe ou módulo faz, quais são seus parâmetros e o que ela retorna.

Dentro da nossa API já podemos ver que algumas funções já possuem docstrings, como por exemplo a função `get_by_email` no arquivo `repositories/author_repository.py`:

```python
def get_by_email(self, email: str) -> AuthorModel | None:
        """Busca um autor pelo email"""
        return self.session.query(AuthorModel).filter(AuthorModel.email == email).first()
```

Entretanto, o ideal é que todas as funções, classes e módulos possuam docstrings. Além disso, note que nesse exemplo falta dizer quais são os parâmetros e o que a função retorna. Vamos melhorar essa docstring:

```python
def get_by_email(self, email: str) -> AuthorModel | None:
        """
        Busca um autor pelo email.

        Args:
            email (str): O email do autor a ser buscado.

        Returns:
            AuthorModel | None: O autor encontrado ou None se nenhum autor for encontrado.
        """
        return self.session.query(AuthorModel).filter(AuthorModel.email == email).first()
```

Agora, tente fazer o mesmo para todas as funções, classes e módulos da nossa API. Lembre-se de descrever claramente o que cada função faz, quais são seus parâmetros e o que ela retorna. Use o padrão de docstrings do Google, como mostrado no exemplo acima. (Aqui não precisa fazer para todas as funcionalidades, mas tente fazer para as principais para pegar o jeito).


### Swagger e OpenAPI

Uma das grandes vantagens do FastAPI é que ele gera automaticamente uma documentação interativa da API usando Swagger UI e Redoc, baseando-se nas rotas, modelos e docstrings que você define no seu código. 

Primeiro, vamos formatar o título e a descrição da nossa API. No arquivo `app.py`, podemos fazer isso ao criar a instância do FastAPI

In [ ]:
# Mude a linha app = FastAPI() para isso:

app = FastAPI(
    title="Documentação da API de livros e autores da Trilha da Insper Jr",
    description="Essa é uma API para gerenciar livros e autores, permitindo operações CRUD e o gerenciamento do relacionamento entre eles.",
    version="1.0.0",
    contact={
        "name": "Guilherme Kaidei",
        "url": "https://blablabla.com",
        "email": "guik@insperjr.com",
    },
    license_info={
        "name": "MIT",
        "url": "https://opensource.org/licenses/MIT",
    },
)

Agora abra o Swagger em `http://localhost:8000/docs` e veja que o título e a descrição da API foram atualizados.

Em seguida, vamos documentar os nossos modelos. No FastAPI, podemos usar o Pydantic para definir nossos modelos de dados, e o FastAPI usa esses modelos para gerar a documentação automaticamente. No entanto, podemos melhorar a documentação dos nossos modelos adicionando descrições aos modelos e seus campos. Por exemplo, no arquivo `entities/author.py`, podemos ver um pouco de como fazer isso:


In [ ]:
from pydantic import BaseModel, EmailStr, Field
from typing import List, Optional

class Author(BaseModel):
    """
    Entidade Author usando Pydantic
    Representa um autor no domínio da aplicação
    """
    id: Optional[int] = None
    name: str = Field(..., min_length=1, max_length=255)
    email: EmailStr
    books: List['Book'] = Field(default_factory=list)

    class Config:
        # Permite conversão de objetos SQLAlchemy
        from_attributes = True

    def __str__(self):
        return f"Author: {self.name} ({self.email})"

# Resolve forward references
from .book import Book
Author.model_rebuild()

Agora abra o Swagger novamente. Perceba que no final da página podemos encontrar a seção de modelos, onde podemos ver a descrição do modelo `Author` e seus campos. 

Nesse caso, você pode ver que com a configuração acima já podemos ver a docstring servindo como descrição do modelo, e os tipos dos campos também aparecem na documentação.

Além disso, podemos adicionar mais coisas nesse modelo, como por exemplo, exemplos de dados e descrições de cada campo. Isso pode ser feito usando os parâmetros `example` e `description` do Pydantic. Veja como fazer isso no arquivo `entities/author.py`:


In [ ]:
from pydantic import BaseModel, EmailStr, Field
from typing import List, Optional
class Author(BaseModel):
    """
    Entidade Author usando Pydantic
    Representa um autor no domínio da aplicação
    """
    id: Optional[int] = Field(None, description="ID do autor", example=1)
    name: str = Field(..., min_length=1, max_length=255, description="Nome do autor", example="João Silva")
    email: EmailStr = Field(..., description="Email do autor", example="joao.silva@example.com")
    books: List['Book'] = Field(default_factory=list, description="Lista de livros do autor")
    
    class Config:
        # Permite conversão de objetos SQLAlchemy
        from_attributes = True

    def __str__(self):
        return f"Author: {self.name} ({self.email})"

# Resolve forward references
from .book import Book
Author.model_rebuild()

Agora, quando você rodar a aplicação e acessar `http://localhost:8000/docs`, verá os campos do modelo estão ainda mais detalhados.

Em seguida, tente fazer o mesmo com os DTOs que criamos para os autores e livros, adicionando descrições, exemplos e qualquer outra informação relevante. Lembre-se de fazer isso tanto para os DTOs de requisição quanto para os DTOs de resposta.

GABARITO:

from pydantic import BaseModel, EmailStr, Field
from typing import List

class CreateAuthorRequest(BaseModel):
    """
    DTO para requisição de criação de autor
    Contém os dados necessários para criar um novo autor
    """
    name: str = Field(..., min_length=1, max_length=255, description="Nome completo do autor", example="João Silva")
    email: EmailStr = Field(..., description="Email válido do autor", example="joao.silva@example.com")

class UpdateAuthorRequest(BaseModel):
    """
    DTO para requisição de atualização de autor
    Contém os dados que podem ser atualizados de um autor
    """
    name: str = Field(..., min_length=1, max_length=255, description="Nome completo do autor", example="João Silva")
    email: EmailStr = Field(..., description="Email válido do autor", example="joao.silva@example.com")

class BookSummary(BaseModel):
    """
    DTO com resumo das informações de um livro
    Usado para evitar referências circulares na resposta de autores
    """
    id: int = Field(..., description="ID único do livro", example=1)
    title: str = Field(..., description="Título do livro", example="Clean Code")
    isbn: str = Field(..., description="ISBN do livro", example="978-0132350884")

    class Config:
        from_attributes = True

class AuthorResponse(BaseModel):
    """
    DTO de resposta para operações com autores
    Contém todas as informações do autor incluindo seus livros
    """
    id: int = Field(..., description="ID único do autor", example=1)
    name: str = Field(..., description="Nome completo do autor", example="João Silva")
    email: str = Field(..., description="Email do autor", example="joao.silva@example.com")
    books: List[BookSummary] = Field(default_factory=list, description="Lista de livros escritos pelo autor")

    class Config:
        from_attributes = True

Tente fazer o mesmo para outros DTOs e entidades para pegar o jeito. Lembre-se de que a documentação deve ser clara e útil para quem for usar a API.

Agora, vamos documentar as nossas rotas. No FastAPI, temos alguns jeitos diferentes de fazer isso.

O primeiro é não escrever nada, assim o FastAPI usa o nome da função como descrição da rota. Por exemplo, no arquivo `use_cases/authors/create_author_use_case/index.py`, temos a seguinte rota:

```python
@router.post("/authors", response_model=Author)
def create_author(request: CreateAuthorRequest, use_case: CreateAuthorUseCase = Depends(get_use_case)):
    return use_case.execute(request.name, request.email)
```

No Swagger, a descrição dessa rota será "Create Author", que é o nome da função. Isso já é um bom começo, mas podemos melhorar isso.

O primeiro jeito de melhorar isso é adicionando uma docstring na função. Por exemplo:



In [ ]:

@router.post("/authors", response_model=Author)
def create_author(request: CreateAuthorRequest, use_case: CreateAuthorUseCase = Depends(get_use_case)):
    """
    Cria um novo autor com o nome e email fornecidos.
    - **name**: Nome do autor
    - **email**: Email do autor
    """
    return use_case.execute(request.name, request.email)


Perceba que a descrição da rota no Swagger é o que está na docstring. Além disso, ao invés de usar uma docstring podemos usar o parâmetro `description` do decorator da rota, como por exemplo:

```python
@router.post("/authors", response_model=Author, description="Cria um novo autor com os dados fornecidos no corpo da requisição.")
def create_author(request: CreateAuthorRequest, use_case: CreateAuthorUseCase = Depends(get_use_case)):
    return use_case.execute(request.name, request.email)
```

Agora, vamos mudar o sumário da rota, que é o texto em negrito que aparece no Swagger. Por padrão, o FastAPI usa o nome da função para isso, mas podemos mudar isso usando o parâmetro `summary` do decorator da rota. Por exemplo:

```python
@router.post("/authors", response_model=Author, description="Cria um novo autor com os dados fornecidos no corpo da requisição.", summary="Criação de Autor")
def create_author(request: CreateAuthorRequest, use_case: CreateAuthorUseCase = Depends(get_use_case)):
    return use_case.execute(request.name, request.email)
```

Além disso temos um último método para mudar esses campos, que é usando um arquivo markdown. Crie uma pasta chamada `docs` dentro da pasta `app`, e dentro dela, crie um arquivo chamado `create_author.md`. Nesse arquivo, você pode escrever a documentação da rota usando markdown. Por exemplo (O swagger não se dá muito bem com acentos e caracteres especiais, então evite usá-los):
```markdown
# Criacao de Autor
## Descricao
Cria um novo autor com os dados fornecidos no corpo da requisição.
## Parametros
- **name** (str): Nome completo do autor. Exemplo: "João Silva"
- **email** (str): Email válido do autor. Exemplo: "joao.silva@example.com"

## Resposta
- **200 OK**: Retorna o autor criado.
- **400 Bad Request**: Retorna um erro se os dados fornecidos forem inválidos.
```

Agora, no arquivo `use_cases/authors/create_author_use_case/index.py`, você pode importar esse arquivo markdown e usá-lo na rota:



In [ ]:
from fastapi import APIRouter, Depends
from repositories.author_repository import AuthorRepository
from database.database import get_db
from sqlalchemy.orm import Session
from entities.author import Author
from use_cases.authors.create_author_use_case.create_author_use_case import CreateAuthorUseCase
from dtos.author_dto import CreateAuthorRequest
from pathlib import Path


router = APIRouter()



def get_use_case(db: Session = Depends(get_db)) -> CreateAuthorUseCase:
    author_repository = AuthorRepository(db)
    return CreateAuthorUseCase(author_repository)

@router.post("/authors", response_model=Author, description=Path("app/docs/create_author.md").read_text(), summary="Criação de Autor")
def create_author(request: CreateAuthorRequest, use_case: CreateAuthorUseCase = Depends(get_use_case)):
    return use_case.execute(request.name, request.email)

Agora, quando você rodar a aplicação e acessar `http://localhost:8000/docs`, verá que o título e a descrição da API foram atualizados.

Por último, um detalhe final é usar tags para organizar as rotas no Swagger. No FastAPI, podemos usar o parâmetro `tags` do decorator da rota para isso. Por exemplo, no arquivo `use_cases/authors/create_author_use_case/index.py`, podemos fazer isso:

```python

```


In [ ]:
@router.post("/authors", response_model=Author, description=Path("app/docs/create_author.md").read_text(), summary="Criação de Autor", tags=["Authors"])
def create_author(request: CreateAuthorRequest, use_case: CreateAuthorUseCase = Depends(get_use_case)):
    return use_case.execute(request.name, request.email)

Assim, podemos agrupar todas as rotas relacionadas a autores sob a tag "Authors" ou de outra categoria que faça sentido. Isso ajuda a organizar a documentação e facilita a navegação para os desenvolvedores que estão consumindo a API.

Agora, tente fazer o mesmo para a maioria das rotas da nossa API, adicionando descrições, sumários, documentação em markdown e tags conforme necessário. Lembre-se de que a documentação deve ser clara e útil para quem for usar a API.

Além disso, caso queira, você pode explorar mais opções de customização da documentação do FastAPI na [documentação oficial](https://fastapi.tiangolo.com/tutorial/metadata/).

Agora chegamos ao final do nosso handout. Parabéns por ter chegado até aqui! 

Você aprendeu como criar uma API RESTful usando FastAPI e SQLAlchemy, seguindo boas práticas de Clean Architecture, incluindo o uso de Use Cases e DTOs para organizar a lógica da aplicação e melhorar a documentação da API. 

O principal objetivo desse handout foi te mostrar como estruturar uma aplicação de forma organizada e escalável, para que você possa aplicar esses conceitos em projetos futuros e melhorar a qualidade dos projetos entregues pela empresa e feitos por você.

### Referências

- [FastAPI Documentation](https://fastapi.tiangolo.com/)
- [SQLAlchemy Documentation](https://docs.sqlalchemy.org/)
- [How to Document an API for Python FastAPI](https://medium.com/codex/how-to-document-an-api-for-python-fastapi-best-practices-for-maintainable-and-readable-code-a183a3f7f036)